In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [17]:
# df_tx = pd.read_csv('15minute_data_austin/15minute_data_austin.csv')
df_ca = pd.read_csv('15minute_data_california/15minute_data_california.csv')
# df_ny = pd.read_csv('15minute_data_newyork/15minute_data_newyork.csv')
df_ca.shape

(805524, 79)

In [18]:
def clean_up_dataset(dataframe):
    sorted_df = dataframe.sort_values(by= ['local_15min'])
    all_sorted_time = sorted_df['local_15min'].unique()
    time_ids = {}
    unique_ids = {}
    counter = 0
    dataframe = dataframe.fillna(0)
    
    # Assign a unique numerical id to each timestamp
    for i in all_sorted_time:
        time_ids.update({i: counter})
        unique_ids.update({counter: i})
        counter += 1

    
    def get_front_back(df, date):
        if time_ids[date] - 1 > 0:
            back = unique_ids[time_ids[date] - 1]
            #while len(df[df['local_15min'] == back]) == 0:
                #back = unique_ids[time_ids[back] - 1]
        else:
            back = date
        if time_ids[date] + 1 < 35035:
            front = unique_ids[time_ids[date] + 1]
            #while len(df[df['local_15min'] == front]) == 0:
                #front = unique_ids[time_ids[front] + 1]
        else:
            front = date
        return (df[df['local_15min'] == front]['grid'], df[df['local_15min'] == back]['grid'])
    
    def missing_times(lst1, lst2):
        # takes unique list and finds the difference in other list
        return list(set(lst1).difference(lst2))
    
    # Append missing data values
    time_labels = []
    grid_interpret = []
    data_labels =[]
    num_uid = 0
    for k in dataframe['dataid'].unique():
        num_uid += 1
        cur_mis = missing_times(all_sorted_time, dataframe[dataframe['dataid'] == k]['local_15min'].unique())
        print(f"Household ID {k} is missing {len(cur_mis)} timestamps.")
        for j in cur_mis:
            #(fbg, cfg) = get_front_back(dataframe[dataframe['dataid'] == k], j)
            time_labels += [j]
            grid_interpret += [np.nan]
            data_labels += [k]
            
    sorted_id = dataframe.append(pd.DataFrame({'local_15min': time_labels, 'grid': grid_interpret, 'dataid': data_labels}),
                    ignore_index = True)
    
    # Resort by data labels
    sorted_id = sorted_id.sort_values(by= ['dataid'])
    sorted_id.reset_index(drop=True,inplace=True)

    # Apply interpolation by indexing into separate UIDs
    df_list = []
    for uid in sorted_id['dataid'].unique():
        grid_pts = sorted_id.loc[sorted_id['dataid']==uid,:].copy()
        grid_pts = grid_pts.sort_values(by=['local_15min'],ignore_index=True)
        interp = grid_pts.interpolate(method='polynomial',order = 5)
        df_list.append(interp)
      
    sorted_id = pd.concat(df_list,ignore_index=True)
    return sorted_id

In [ ]:
# sorted_cleaned_tx = clean_up_dataset(df_tx)
sorted_cleaned_ca = clean_up_dataset(df_ca)
# sorted_cleaned_ny = clean_up_dataset(df_ny)

Household ID 3687 is missing 105177 timestamps.
Household ID 6377 is missing 105177 timestamps.
Household ID 7062 is missing 105177 timestamps.
Household ID 8574 is missing 105177 timestamps.
Household ID 9213 is missing 105177 timestamps.
Household ID 203 is missing 105177 timestamps.
Household ID 1450 is missing 105177 timestamps.
Household ID 1524 is missing 105177 timestamps.
Household ID 2606 is missing 105177 timestamps.
Household ID 3864 is missing 105177 timestamps.
Household ID 7114 is missing 105177 timestamps.
Household ID 1731 is missing 105223 timestamps.
Household ID 4495 is missing 105212 timestamps.
Household ID 8342 is missing 105208 timestamps.
Household ID 3938 is missing 105081 timestamps.
Household ID 5938 is missing 105081 timestamps.
Household ID 8061 is missing 105081 timestamps.
Household ID 9775 is missing 105081 timestamps.
Household ID 4934 is missing 105081 timestamps.
Household ID 8733 is missing 105177 timestamps.
Household ID 9612 is missing 105177 times

In [14]:
sorted_cleaned_ca

,dataid,local_15min,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,...,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,leg1v,leg2v
0,27,2019-05-01 00:00:00-05,0.693,0.054,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.000,0.0,117.873,117.992
1,27,2019-05-01 00:15:00-05,0.554,0.006,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.000,0.0,118.538,118.602
2,27,2019-05-01 00:30:00-05,0.393,0.012,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.000,0.0,118.377,118.454
3,27,2019-05-01 00:45:00-05,0.801,0.060,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.000,0.0,118.634,118.695
4,27,2019-05-01 01:00:00-05,0.237,0.053,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.001,0.0,0.000,0.0,118.820,118.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441595,9053,2019-10-31 22:45:00-05,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.464,0.0,-0.003,0.0,118.951,119.321
441596,9053,2019-10-31 23:00:00-05,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.472,0.0,-0.002,0.0,119.217,119.278
441597,9053,2019-10-31 23:15:00-05,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.479,0.0,-0.002,0.0,119.684,119.765
441598,9053,2019-10-31 23:30:00-05,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.487,0.0,-0.002,0.0,120.851,120.973
